# Поиск по текстовым полям SQLite с учётом морфологии

**ЗАДАЧА:** Нужно реализовать сервис, которые будет отдавать данные о загруженных файлах. Есть готовая БД на SQLite, в которую загружают новый контент через админку. Поле **title** и **description** это текстовое поле. Поле **path** это ссылка на файл. Решение должно хорошо горизонтально масштабироваться.
На данном этапе отказаться от SQLite нельзя, но нужно придумать решение для поиска по тексту и описанию загруженных файлов с учётом морфологии. Текст может быть на английском или русском языке.

Рассмотрим базу данных строительных материалов. В ней одна таблица - **files**. Поле **id** - первичный ключ, поля **title** и **description** содержат название и описание товара соответственно, а в поле **path** хранится путь к его изображению.

In [2]:
import sqlite3
db_file = "files.db"
conn = sqlite3.connect(db_file)
c = conn.cursor()

## 1. Поиск вхожений с помощью оператора LIKE
Самое очевидное решение: использовать оператор LIKE.
Попробуем найти в описании файлов слово **"пол"**:

In [3]:
c.execute('''SELECT * FROM files WHERE description LIKE "%пол%"''')
c.fetchall()[:2]

[(2,
  'bag.jpg',
  'Сеточная корзина 590x410x85',
  'Предназначена для хранения вещей, инструментов и различных аксессуаров. Устанавливается на кронштейны для корзин: двухсторонние и комплект системы хранения. Прочная, простая в уходе и устойчивая к загрязнениям поверхность. Можно также использовать вванной и других помещениях с повышенным уровнем влажности. При появлении загрязнений просто протрите мягкой влажной тканью, при необходимости можно использовать слабый мыльный раствор.'),
 (4,
  'ondulin.jpg',
  'Ондулин SMART, цвет коричневый, 1950 х 950 мм',
  'При монтаже умный замок работает как направляющие и фиксирует листы так, чтобы получить идеально ровный край.')]

Это выражение вернёт все записи, в поле *description* которых содержится слово **"пол"**. А также эти слова:

у**пол**номоченный

**пол**овина

**пол**ка

**пол**укруглый

Проблема подхода с использованием LIKE не только в ложных срабатываниях и невысокой скорости выполнения (нужно проверить все записи, никакого индексирования нет). Можно требовать точного совпадения, предварительно разбив текст на слова (этот процесс называется *токенизацией* по словам) и сравнивая слова с образцом целиком.

***ЗАДАНИЕ 1:*** *реализуйте простой токенизатор, разбивающий текст на список слов с учётом знаков препинания с помощью функции split().*

Но если при поиске, к примеру, слова **"гвозди"** мы захотим добавить в выборку и строки, содержащие различные формы этого слова - **гвоздей**, **гвоздь**, **гвоздями** (то есть, с учётом *морфологии* слов), такой подход работать не будет.

## 2. Поиск с учётом морфологии

Для реализации поиска с учётом морфологии нужно привести слова исходного текста и поискового запроса к некой канонической (*нормальной*) форме. Два наиболее распространённых подхода к получению этих форм - это **стемминг** и **лемматизация**.

При **стемминге** от слов просто отсекается определённая часть (например, популярный в английском языке стеммер Портера отексает окончания и суффиксы).

одежды - одежд

гвозди - гвозд

ответственность - отвествен

Алгоритмы стемминга работают быстро, но не всегда точно.


**Лемматизация** приводит слово к нормальной форме по правилам языка (например, для существительных русского языка это единственное число и именительный падеж).

гвоздей - гвоздь

убегали - убегать

белых - белый

Лемматизация требует наличия словаря, работает медленнее, но заметно точнее.

### Как это реализовать

Можно использовать готовые библиотеки для обработки языка (такие как [NLTK](https://www.nltk.org/) и [spaCy](https://spacy.io/)).В них уже встроены механизмы стемминга и лемматизации. Можно даже подключить полноценную поисковую систему ([Elasticsearch](https://www.elastic.co/elasticsearch/), [Sphinx](http://sphinxsearch.com/) или [Solr](https://lucene.apache.org/solr/)). Но наш проект на SQLite, а значит решение должно быть встраиваемым, простым, легковесным и, по возможности, быстрым.

### Возможности SQLite

В SQLite есть расширение **FTS** (*Full-Text Search, полнотекстовый поиск*), в которое встроен стеммер Портера, правда только для английского языка. Чтобы добавить русский язык, воспользуемся стеммером [Snowball](https://snowballstem.org/), в котором есть поддержка русского языка. Поэкспериментруйте [c онлайн версией](https://snowballstem.org/demo.html) Snowball, чтобы понять принцип его работы (можно менять языки).

Описание алгоритмов стемминга, использующихся в Snowball для [русского](https://snowballstem.org/algorithms/russian/stemmer.html) и
[английского](https://snowballstem.org/algorithms/porter/stemmer.html) языков.

Для подключения Snowball к SQLite (версии 3.20.0) нужно установить [расширение](https://github.com/abiliojr/fts5-snowball) (компилируется из исходников).

In [4]:
# Разрешаем подключать расширения к SQLite
conn.enable_load_extension(True)
# Подключаем расширение 'fts5stemmer'
conn.load_extension("fts5stemmer")

Для работы FTS нужно создать т.н. *виртуальную* таблицу, куда заносятся данные, участвующие в поиске. Поэтому в нашу виртуальную таблицу *f_search* поле **path** не добавляем:

In [5]:
c.execute('''DROP TABLE IF EXISTS f_search''')
c.execute('''CREATE VIRTUAL TABLE f_search USING fts5(title, description,              
              content_rowid='id',
              tokenize = 'snowball russian english')''')
conn.commit()

In [6]:
c.execute('''INSERT INTO f_search SELECT title,description from files''')
conn.commit()

Аргумент *tokenize* - указывает какой токенизатор использовать (в нашем случае это *snowball* и параметры *russian* и *english* для подержки русского и английского языков соответственно).

SQLite вместе с таблицей *f_search* создаёт ещё несколько служебных таблиц для хранения индексов. Нам к ним обращаться не нужно - они нужны для использования внутри SQLite.

При добавлении новых полей для поиска необходимо обновить индексную таблицу.

**ВНИМАНИЕ!** Для обновления индексов виртуальной таблицы при удалении, вставке или изменении элементов базовой необходимо добавить триггеры на события AFTER DELETE, AFTER INSERT и AFTER UPDATE ([инструкция](https://www.sqlite.org/fts5.html#external_content_tables)).

Перейдём к поиску. Вместо LIKE используем оператор MATCH

In [10]:
c.execute('''SELECT * FROM f_search WHERE description MATCH ?''', ['пол'])
c.fetchall()[:2]

[('Теплый пол нагревательный мат ГдеМорозаНет 1м2, 160Вт',
  'Нагревательные маты ГдеМорозаНет состоят из двухжильного нагревательного экранированного кабеля, укрепленного на монтажной сетке. Маты поставляются в виде рулонов: ширина мата составляет 500 мм, длина от 1 до 20 м. Широкий выбор типоразмеров от 0,5 кв. м до 10 кв. м. позволяет подобрать оптимальный размер <теплого пола> для любого помещения.'),
 ('Наливной пол тонкий Старатели, 25 кг',
  'Ровнитель для пола Тонкий, 25 кг Старатели')]

Поддерживаются и логические операции:

In [11]:
c.execute('''SELECT * FROM f_search WHERE description MATCH ?''', ['пол AND теплый'])
c.fetchall()[:2]

[('Теплый пол нагревательный мат ГдеМорозаНет 1м2, 160Вт',
  'Нагревательные маты ГдеМорозаНет состоят из двухжильного нагревательного экранированного кабеля, укрепленного на монтажной сетке. Маты поставляются в виде рулонов: ширина мата составляет 500 мм, длина от 1 до 20 м. Широкий выбор типоразмеров от 0,5 кв. м до 10 кв. м. позволяет подобрать оптимальный размер <теплого пола> для любого помещения.'),
 ('Комплект теплый пол Thermo SVK-20 8м без регулятора',
  'Thermocable - греющий кабель. Теплый пол - самый комфортный способ держать ноги в тепле. Греющий кабель – элемент, на котором основан принцип действия любой системы электрообогрева. Его роль заключается в преобразовании протекающего по нему тока в тепло. Поэтому мощность на единицу длины (удельное тепловыделение, Вт/м) – главная техническая характеристика нагревательного кабеля. Основное применение греющего кабеля Thermocable - обогрев пола. Он укладывается в цементную стяжку и служит десятилетиями. Используя кабельные систем

In [12]:
c.execute('''SELECT * FROM f_search WHERE description MATCH ?''', ['пол OR теплый'])
c.fetchall()[:2]

[('Теплоизоляция ISOVER Тёплый Дом',
  'ISOVER ТЕПЛЫЙ ДОМ ПЛИТА - тепло- и звукоизоляция из минеральной ваты. Материал производится из природных компонентов: песок, сода, известняк. Является безопасным материалом для здоровья человека и окружающей среды: сертифицирован для применения в детских и медицинских учреждениях. Эффективное решение для всестороннего утепления и звукоизоляции дома одним продуктом.'),
 ('Теплый пол нагревательный мат ГдеМорозаНет 1м2, 160Вт',
  'Нагревательные маты ГдеМорозаНет состоят из двухжильного нагревательного экранированного кабеля, укрепленного на монтажной сетке. Маты поставляются в виде рулонов: ширина мата составляет 500 мм, длина от 1 до 20 м. Широкий выбор типоразмеров от 0,5 кв. м до 10 кв. м. позволяет подобрать оптимальный размер <теплого пола> для любого помещения.')]

In [14]:
c.execute('''SELECT * FROM f_search WHERE title MATCH ?''', ['пол NOT теплый'])
c.fetchall()[:2]

[('Пол ламин. Kastamonu RED 27 Дуб Сенегал',
  'Коллекция: Floorpan Red\ufeff (Флорпеин Ред)'),
 ('Наливной пол тонкий Старатели, 25 кг',
  'Ровнитель для пола Тонкий, 25 кг Старатели')]

In [15]:
conn.close()

***ЗАДАНИЕ 2:*** *Изучите возможности оператора NEAR из [документации FTS](https://www.sqlite.org/fts5.html). Попробуте запустить с ним несколько запросов. Может ли он использоваться для нашей базы данных?*

***ЗАДАНИЕ 3:*** *Скомбинируйте поиск по полям title и description в один запрос.*

***ЗАДАНИЕ 4:*** *Как можно выделять вхождения текста (текстом, тегами HTML или другим способом)?*

***ЗАДАНИЕ 5:*** *Подумайте над ранжированием резульатов поиска.*

Теперь поиск поддерживает морфологию слов на уровне несложного стемминга. Однако, даже это значительно улучшило поисковые возможности для нашей базы данных.

Полезные ссылки:

1. [Основы Natural Language Processing для текста](https://habr.com/ru/company/Voximplant/blog/446738/).
2. [Полнотекстовый поиск в SQLite](http://blog-programmista.ru/post/45-polnotekstovyj-poisk-v-sqlite.html)
3. [Документация FTS5 (англ.)](https://www.sqlite.org/fts5.html)